In [7]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JLD2, FileIO
using JuMP

  Activating project at `~/Documents/MacroEnergy.jl`


In [ ]:
import Pkg; Pkg.add("Plots")

   Resolving package versions...
   Installed LERC_jll ───────────────────── v4.0.1+0
   Installed GR_jll ─────────────────────── v0.73.16+0
   Installed Opus_jll ───────────────────── v1.3.3+0
   Installed libfdk_aac_jll ─────────────── v2.0.3+0
   Installed libdecor_jll ───────────────── v0.2.2+0
   Installed JpegTurbo_jll ──────────────── v3.1.1+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed Libmount_jll ───────────────── v2.41.0+0
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Xorg_xkbcomp_jll ───────────── v1.4.7+0
   Installed Grisu ──────────────────────── v1.0.2
   Installed Measures ───────────────────── v0.3.2
   Installed Unitful ────────────────────── v1.23.1
   Installed Contour ────────────────────── v0.6.3
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.2+0
   Installed Xorg_xcb_util_image_jll ────── v0.4.1+0
   Installed DelimitedFiles ─────────────── v1.9.1
   Installed RecipesPipeline ────────────── v0.6.12
   Installed PlotUtils

In [4]:
import Pkg; Pkg.add("JLD2")

   Resolving package versions...
   Installed JLD2 ─── v0.5.13
   Installed FileIO ─ v1.17.0
      Compat entries added for JLD2
    Updating `~/Documents/MacroEnergy.jl/Project.toml`
  [033835bb] + JLD2 v0.5.13
    Updating `~/Documents/MacroEnergy.jl/Manifest.toml`
  [5789e2e9] + FileIO v1.17.0
  [033835bb] + JLD2 v0.5.13
Precompiling project...
   1340.7 ms  ✓ FileIO
    780.6 ms  ✓ FileIO → HTTPExt
   1684.0 ms  ✓ Plots → FileIOExt
   9618.1 ms  ✓ JLD2
  4 dependencies successfully precompiled in 11 seconds. 303 already precompiled.


In [6]:
import Pkg; Pkg.add("FileIO")

   Resolving package versions...
      Compat entries added for FileIO
    Updating `~/Documents/MacroEnergy.jl/Project.toml`
  [5789e2e9] + FileIO v1.17.0
  No Changes to `~/Documents/MacroEnergy.jl/Manifest.toml`


In [9]:
include("helper_functions.jl");

In [11]:
system = MacroEnergy.load_system(@__DIR__)
model = MacroEnergy.generate_model(system)

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/eastern_us_three_zones_with_cement/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:39
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194


A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 727126
├ num_constraints: 1725790
│ ├ AffExpr in MOI.EqualTo{Float64}: 385441
│ ├ AffExpr in MOI.GreaterThan{Float64}: 78840
│ ├ AffExpr in MOI.LessThan{Float64}: 551903
│ ├ VariableRef in MOI.EqualTo{Float64}: 13
│ └ VariableRef in MOI.GreaterThan{Float64}: 709593
└ Names registered in the model
  └ :cRetrofitCapacity, :eFixedCost, :eVariableCost, :vREF

In [12]:
MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
MacroEnergy.optimize!(model)

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 1016184 rows, 727126 columns and 3257682 nonzeros
Model fingerprint: 0xff6fda8d
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 4e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+04]
Presolve removed 534916 rows and 347673 columns
Presolve time: 0.64s
Presolved: 481268 rows, 379453 columns, 2292429 nonzeros
Ordering ti

In [13]:
case = "0pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results")
mkpath(results_dir)
save_results(results_dir, case, system)

"/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/eastern_us_three_zones_with_cement/results/0pct_co2_reduction_flows.csv"

In [3]:
# Run model for different CO2 caps, if you have a no_cap_co2_level

no_cap_co2_level = 1.484e6
co2_reductions = [0.2, 0.4, 0.6, 0.8] # If there is a 0, it must be the first value
system = nothing
model = nothing

for co2_reduction in co2_reductions
    # Load system
    case_path = @__DIR__
    println("###### ###### ######")
    println("Running case at $(case_path)")
    system = MacroEnergy.load_system(case_path)

    # Get the CO2 node
    nodes = Node[x for x in system.locations if x isa MacroEnergy.Node]
    co2_node = MacroEnergy.get_nodes_sametype(nodes, CO2)[1] # the cement_co2_sink node is the first CO2 node

    if co2_reduction == 0 # If there is no CO2 cap
        case = "0pct_co2_reduction"
        co2_node.constraints = AbstractTypeConstraint[] # Remove CO2 cap constraint
    else # If there is not a CO2 cap
        case = string(floor(Int, co2_reduction * 100)) * "pct_co2_reduction"
        co2_node.rhs_policy[CO2CapConstraint] = no_cap_co2_level * (1 - co2_reduction) # Set CO2 cap
        print(no_cap_co2_level * (1 - co2_reduction))
    end

    # Run model
    model = MacroEnergy.generate_model(system)
    MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
    MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
    MacroEnergy.optimize!(model)

    # Save results
    results_dir = joinpath(case_path, "results")
    mkpath(results_dir)
    save_results(results_dir, case, system)
    # @save case * "_results.jld2" system_results_df capacity_results_df results_8760_df
    # println("wrote results")

    # Record the no_cap_co2_level for next iterations
    if co2_reduction == 0
        no_cap_co2_level = MacroEnergy.value(sum(co2_node.operation_expr[:emissions]))
    end
end

###### ###### ######
Running case at /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test


┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.32 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ W

1.1872e6

┌ Info: Generating model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:3
┌ Info:  -- Adding linking variables
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:15
┌ Info:  -- Defining available capacity
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:18
┌ Info:  -- Generating planning model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:21
┌ Info:  -- Generating operational model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:24
┌ Info:  -- Model generation complete. It took 11.26 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:29


Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 840991 rows, 551927 columns and 2845967 nonzeros
Model fingerprint: 0x3acb817f
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+06]
Presolve removed 394763 rows and 189997 columns
Presolve time: 0.49s
Presolved: 446228 rows, 361930 columns, 2196063 nonzeros
Ordering ti

...

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 840991 rows, 551927 columns and 2845967 nonzeros
Model fingerprint: 0xa9aae9b6
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 6e+05]
Presolve removed 394763 rows and 189997 columns
Presolve time: 0.49s
Presolved: 446228 rows, 361930 columns, 2196063 nonzeros
Ordering ti

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_locations_test/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:17
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:25
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: TotalHoursModeled not found in time_data.json - Using 8760 as default value for TotalHoursModeled
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:79
┌ Warning: Using default weights = 1 as no period map provided and each period maps to itself
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:194
┌ Wa

296799.99999999994

┌ Info: Generating model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:3
┌ Info:  -- Adding linking variables
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:15
┌ Info:  -- Defining available capacity
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:18
┌ Info:  -- Generating planning model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:21
┌ Info:  -- Generating operational model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:24
┌ Info:  -- Model generation complete. It took 6.9 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:29


Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 840991 rows, 551927 columns and 2845967 nonzeros
Model fingerprint: 0x3162dcbc
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 3e+05]
Presolve removed 394763 rows and 189997 columns
Presolve time: 0.51s
Presolved: 446228 rows, 361930 columns, 2196063 nonzeros
Ordering ti